<a href="https://colab.research.google.com/github/er-ay/Feature_engineering/blob/main/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

Makine öğrenmesinde modelin iyi öğrenmesini sağlamak, yüksek doğruluğa sahip çıktı almak modelin eğitildiği verinin ne kadar iyi işlendiğine bağlıdır. Veri setleri farklı veri tipleri içeren, eksik girilmiş veya farklı aralıklarda sayısal değere sahip kolonlardan oluşabilir. Bu durum, ağın doğru ilişkiyi kuramamasına dolayısıyla öğrenmeyi gerçekleştirememesine sebep olur. Bu sorunu ortadan kaldırmak için çeşitli yöntemler kullanılır. Python dilinde bu işlemleri gerçekleştirmek için geliştirilmiş Pandas kütüphanesi büyük kolaylık sağlar.

Veri tipleri:
+ Categorical
+ Ordinal
+ Continious
+ Datetime
+ Boolean

## Pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head() #Verinin ilk 5 satırına göz atmak için
df.columns #Kolonları yazdırır.
df.dtypes #Veri tiplerini yazdırır
df.select_dtypes(include=['dtype']) #Belirli bir veri tipi seçer. dtype int,float vb. olabilir

## Encoding

Gerçek hayatta insanların anlayabileceği kategorik verileri (renk,yaş,ülke vb.) bilgisayarın anlayabileceği nümerik verilere dönüştürme işlemine encoding denir.

In [ ]:
d = {'Meyve' : ['Elma','Armut','Muz','Muz','Üzüm','Armut']}
veri = pd.DataFrame(data=d)
veri

,Meyve
0,Elma
1,Armut
2,Muz
3,Muz
4,Üzüm
5,Armut


Bu verideki meyveleri nümerik verilere çevirmek için encoding yapabiliriz. Her meyveye sayısal bir değer atamak, kategorik verinin birinin diğerinden daha büyük değere sahip olması şeklinde öğrenileceği için doğru bir yöntem olmayacaktır.
Elma = 1, Armut = 2 => Armut > Elma gibi.
Bunun için her özelliğin o satırda seçilip seçilmediğini gösterecek binary şekilde kodlanır. Bunun için 2 farklı kodlama yapılabilir:

1. One-hot encoding
2. Dummy encoding

2 yöntemde pandasta aynı şekilde çağrılır.

In [ ]:
pd.get_dummies(veri, columns=['Meyve'], prefix='M')

,M_Armut,M_Elma,M_Muz,M_Üzüm
0,0,1,0,0
1,1,0,0,0
2,0,0,1,0
3,0,0,1,0
4,0,0,0,1
5,1,0,0,0


One-hot encoding n kategori için n kodlama yapar. get_dummies fonksiyonu ile one-hot encoding gerçekleştirilir. drop_first metodu eklenerek ise dummies encoding sağlanır. Dummies encoding n özellik için n-1 kodlama yapar ve kodlanmayan 1 seçenek kodlananların hiçbiri olmayan (hepsinin 0 olduğu) diğer durumu ifade eder.

In [ ]:
pd.get_dummies(veri, columns=['Meyve'], drop_first=True, prefix='M')

,M_Elma,M_Muz,M_Üzüm
0,1,0,0
1,0,0,0
2,0,1,0
3,0,1,0
4,0,0,1
5,0,0,0


Özelliklerin sayısını görmek için:

In [ ]:
counts = veri['Meyve'].value_counts()
counts

Armut    2
Muz      2
Üzüm     1
Elma     1
Name: Meyve, dtype: int64

In [ ]:
Özellikleri sınırlamak için:

In [ ]:
mask = veri['Meyve'].isin(counts[counts < 2].index)
mask

0     True
1    False
2    False
3    False
4     True
5    False
Name: Meyve, dtype: bool

In [ ]:
veri[mask] = 'Other'
veri['Meyve'].value_counts()

Muz      2
Other    2
Armut    2
Name: Meyve, dtype: int64

Veri setinde 2den az bulunan özellikler (Elma,Üzüm) maskeleyip onların sayısının 'Other' şeklinde ifade ettik. Bu şekilde veri setinde belli bir sayıdan az, önem ifade etmeyen elemanları maskelemiş olduk.

### Nümerik değişkenleri binary ifade etmek (Binarizing)

Öğrenci devamsızlıklarından oluşan bir veri seti olsun. Belirli sayıda devamsızlık yapmış olan öğrencileri belirlemek isteyelim.

In [ ]:
data = { 'ogrenci':['Ali','Veli','Ahmet','Ayse','Merve','Betul'], 'devamsizlik':[1,3,2,10,2,4]}
ogr_dev = pd.DataFrame(data=data)
ogr_dev

,ogrenci,devamsizlik
0,Ali,1
1,Veli,3
2,Ahmet,2
3,Ayse,10
4,Merve,2
5,Betul,4


Yeni bir kolon oluşturup devamsızlığı 5 günden fazla olan öğrencileri binary olarak ifade edelim. Başlangıç olarak tüm değerler 0.

In [ ]:
ogr_dev['uyari'] = 0

### Veri setinden alt grup seçimi: loc

In [ ]:
ogr_dev.loc[ogr_dev['devamsizlik'] > 5, 'uyari'] = 1
ogr_dev

,ogrenci,devamsizlik,uyari
0,Ali,1,0
1,Veli,3,0
2,Ahmet,2,0
3,Ayse,10,1
4,Merve,2,0
5,Betul,4,0


### Aralıklara bölme (Binning)

Bu sefer devamsızlıkları belirli aralıklarla ifade edelim ve 5i geçmemiş ama yaklaşanları da belirleyelim.

Pandas ile binning denilen bu işlemi yapmak için cut fonksiyonu kullanılır.

In [ ]:
import numpy as np

In [ ]:
ogr_dev['Binned_uyari'] = pd.cut(ogr_dev['devamsizlik'], bins=[0, 3, 5, np.inf], labels=['-','sinirda','siniri_asmis'])
ogr_dev

,ogrenci,devamsizlik,uyari,Binned_uyari
0,Ali,1,0,-
1,Veli,3,0,-
2,Ahmet,2,0,-
3,Ayse,10,1,siniri_asmis
4,Merve,2,0,-
5,Betul,4,0,sinirda


pd.cut fonksiyonunda bins ile aralıkları belirledik. 
0-3 için -
3-5 için sinirda
5+ için siniri_asmis

### Kayıp veriler

Gerçek dünya verileri genellikle kullanıma hazır durumda değildir. Verinin uygun şekilde toplanmamış olması, eksik girilmiş olması, elle giriliyorsa basit bir klavye hatası gibi sebeplerle verilerin eğitime hazır hale getirilmesi gerekir. info() fonksiyonunu kullanarak verinin kolonlarındaki non-null entry sayısını görüntülenebilir.

In [ ]:
data = pd.read_csv('...heart.csv')

In [ ]:
data

,age,sex,chol
0,63.0,1,233.0
1,37.0,1,NaN
2,41.0,0,204.0
3,NaN,1,236.0
4,57.0,0,354.0
5,57.0,1,NaN
6,NaN,0,294.0
7,44.0,1,263.0
8,52.0,1,199.0
9,57.0,1,168.0


Bu data, kalp hastalıklarıyla ilgili bir veri setinin küçük bir bölümünü göstermektedir.
age = yaş
sex = cinsiyet
chol = kolestrol
Veri seti normalde NaN değer içermiyor, örnek olması açısından bazı değerleri sildim.
NaN = Not a Number

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     8 non-null      float64
 1   sex     10 non-null     int64  
 2   chol    8 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 368.0 bytes


Veri setini incelediğimizde 10 indeksli olduğunu, 'age' kolonunda 8, 'chol' kolonunda 8 non-null girdi olmasından da bazı girdilerin eksik olduğunu görüyoruz. Bu eksik değerlerin yerlerini veri setinde görmek için isnull() kullanılır.

In [ ]:
data.isnull()

,age,sex,chol
0,False,False,False
1,False,False,True
2,False,False,False
3,True,False,False
4,False,False,False
5,False,False,True
6,True,False,False
7,False,False,False
8,False,False,False
9,False,False,False


is null sorguladığımız için True olan değerler bize eksik verileri gösterecektir. Eğer eksik verilerin toplamı görülmek isteniyorsa:

In [ ]:
data.isnull().sum()

age     2
sex     0
chol    2
dtype: int64

In [ ]:
data['chol'].isnull().sum()

2

Aynı şekilde null olmayan girdiler de sorgulanabilir:

In [ ]:
data.notnull()

,age,sex,chol
0,True,True,True
1,True,True,False
2,True,True,True
3,False,True,True
4,True,True,True
5,True,True,False
6,False,True,True
7,True,True,True
8,True,True,True
9,True,True,True


Eksik verileri tespit ettikten sonra bunlarla ilgili neler yapılabileceğine sıra geldi.

### Eksik değerleri silme

In [ ]:
data.dropna(how='any')

,age,sex,chol
0,63.0,1,233.0
2,41.0,0,204.0
4,57.0,0,354.0
7,44.0,1,263.0
8,52.0,1,199.0
9,57.0,1,168.0


Bu işlem belirli bir kolona da uygulanabilir.

In [ ]:
data.dropna(subset=['chol'])

,age,sex,chol
0,63.0,1,233.0
2,41.0,0,204.0
3,NaN,1,236.0
4,57.0,0,354.0
6,NaN,0,294.0
7,44.0,1,263.0
8,52.0,1,199.0
9,57.0,1,168.0


Sadece 'chol' kolonunda eksik veri bulunduran satırlar silindi.

In [ ]:
data

,age,sex,chol
0,63.0,1,233.0
1,37.0,1,NaN
2,41.0,0,204.0
3,NaN,1,236.0
4,57.0,0,354.0
5,57.0,1,NaN
6,NaN,0,294.0
7,44.0,1,263.0
8,52.0,1,199.0
9,57.0,1,168.0


Ancak veriye bakarsak NaN değerleri içerdiğini görürüz. Yaptığımız bu değişikliği veri setine kalıcı olarak uygulamak için inplace='True' eklemek gerekir. Default olarak False'dur.

In [ ]:
data.dropna(how='any', inplace=True)
data

,age,sex,chol
0,63.0,1,233.0
2,41.0,0,204.0
4,57.0,0,354.0
7,44.0,1,263.0
8,52.0,1,199.0
9,57.0,1,168.0


### Eksik değerleri doldurma

Veri setindeki eksik değerleri silmenin yanı sıra uygun değerlerle doldurmak da doğru bir yöntem olabilir. Örneğin, sayısal değer içeren kolonlar için, eksik değeri o kolonun ortalaması olan sayıyla doldurabiliriz. Kolonda eksik veri oranı az ise bu iyi bir seçim olabilir.

In [ ]:
data['chol'].fillna(value=data['chol'].mean(), inplace=True)
data['chol']

0    233.000
1    243.875
2    204.000
3    236.000
4    354.000
5    243.875
6    294.000
7    263.000
8    199.000
9    168.000
Name: chol, dtype: float64

Ortalama değer, ondalıklı sayı olabilir ve eğer bu istenmeyen bir durumsa 2 şekilde düzeltilebilir:
-Veri tipi integer olarak değiştirilerek
-Ortalama değeri yuvarlayarak

In [ ]:
data['chol'].astype('int64')

0    233
1    244
2    204
3    236
4    354
5    244
6    294
7    263
8    199
9    168
Name: chol, dtype: int64

In [ ]:
x = round(data['chol'].mean())
x

244

In [ ]:
data['chol'].fillna(value=x, inplace=True)
data['chol']

0    233.0
1    244.0
2    204.0
3    236.0
4    354.0
5    244.0
6    294.0
7    263.0
8    199.0
9    168.0
Name: chol, dtype: float64

### Karakter değiştirme

Veri setinde değiştirilmek istenen karakter için str.replace() kullanılır. Örnek olarak aşağıdaki veride Ayse Nur olarak yazılması gerekek girdi Ayse,Nur olarak yazılmış. 

In [ ]:
veri = { 'ad':['Ali','Veli','Ahmet','Ayse,Nur','Merve','Betul'], 'devamsizlik':[1,3,2,10,2,4]}
ogrenci = pd.DataFrame(data=veri)
ogrenci

,ad,devamsizlik
0,Ali,1
1,Veli,3
2,Ahmet,2
3,"Ayse,Nur",10
4,Merve,2
5,Betul,4


In [ ]:
ogrenci = ogrenci['ad'].str.replace(',',' ') 
# değiştirilmek istenen yerine boşluk bıraktım. Virgülün kaldırılması istenen durumlar için sadece '' konulmalıdır.

In [ ]:
ogrenci

0         Ali
1        Veli
2       Ahmet
3    Ayse Nur
4       Merve
5       Betul
Name: ad, dtype: object